In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ, ClassicalRegister, QuantumRegister
from qiskit.compiler import transpile, assemble
import numpy as np
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)

In [2]:
def get_block_index(x, y):
    if x<2 and y<2:
        return 0
    if x<2:
        return 1
    if y<2:
        return 2
    return 3

def get_cands(mat):
    cands = []
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            if mat[i][j] != 0:
                continue
            # (i, j) に k を置けるか
            tmp = []
            for k in range(4):
                f = True
                for row in range(4):
                    if mat[row][j] == k + 1:
                        f = False
                for col in range(4):
                    if mat[i][col] == k + 1:
                        f = False
                for x in range(4):
                    for y in range(4):
                        if get_block_index(i, j) != get_block_index(x, y):
                            continue
                        if mat[x][y] == k + 1:
                            f = False
                if f:
                    tmp.append(k+1)
            cands.append(tmp)
    return cands

In [3]:
def single_cand_operation(qc, q0, q1, cands):
    # print(q0, q1)
    if cands[0] == 1:
        pass
    elif cands[0] == 2:
        qc.x(q1)
    elif cands[0] == 3:
        qc.x(q0)
    else:
        qc.x(q0)
        qc.x(q1)

def two_cand_operation(qc, q0, q1, cands):
    # print(q0, q1, cands[0], cands[1])
    product = cands[0]*cands[1]
    #1 + 2
    if product == 2:
        qc.h(q1)
    #1 + 3
    elif product == 3:
        qc.h(q0)
    #1 + 4
    elif product == 4:
        qc.h(q0)
        qc.cx(q0, q1)
    #2 + 3
    elif product == 6:
        qc.x(q1)
        qc.h(q0)
        qc.cx(q0, q1)
    #2 + 4
    elif product == 8:
        qc.x(q1)
        qc.h(q0)
    #3 + 4
    else:
        qc.h(q1)
        qc.x(q0)

In [4]:
def single_cand_operation_inv(qc, q0, q1, cands): 
    '''identical as noinv'''
    if cands[0] == 1:
        pass
    elif cands[0] == 2:
        qc.x(q1)
    elif cands[0] == 3:
        qc.x(q0)
    else:
        qc.x(q1)
        qc.x(q0)
    
def two_cand_operation_inv(qc, q0, q1, cands):
    product = cands[0]*cands[1]
    #1 + 2
    if product == 2:
        qc.h(q1)
    #1 + 3
    elif product == 3:
        qc.h(q0)
    #1 + 4
    elif product == 4:
        qc.cx(q0, q1)
        qc.h(q0)
    #2 + 3
    elif product == 6:
        qc.cx(q0, q1)
        qc.h(q0)
        qc.x(q1)
    #2 + 4
    elif product == 8:
        qc.h(q0)
        qc.x(q1)
    #3 + 4
    else:
        qc.h(q1)
        qc.x(q0)

In [5]:
import math

def three_cand_operation(qc, q0, q1, cands):
    product = cands[0]*cands[1]
    theta = 2*math.acos(1/math.sqrt(3))
    #1 + 2 + 3
    if product == 6:
        qc.ry(theta, q0)
        qc.ch(q0, q1)
        qc.x(q0)
    #1 + 2 + 4
    elif product == 8:
        qc.ry(theta, q0)
        qc.ch(q0, q1)
    #1 + 3 + 4
    elif product == 12:
        qc.ry(theta, q1)
        qc.ch(q1, q0)
    #2 + 3 + 4
    else:
        qc.ry(theta, q0)
        qc.ch(q0, q1)
        qc.x(q1)
    
def three_cand_operation_inv(qc, q0, q1, cands):
    product = cands[0]*cands[1]
    theta = 2*math.acos(1/math.sqrt(3))
    #1 + 2 + 3
    if product == 6:
        qc.x(q0)
        qc.ch(q0, q1)
        qc.ry(theta, q0)
    #1 + 2 + 4
    elif product == 8:
        qc.ch(q0, q1)
        qc.ry(theta, q0)
    #1 + 3 + 4
    elif product == 12:
        qc.ch(q1, q0)
        qc.ry(theta, q1)
    #2 + 3 + 4
    else:
        qc.x(q1)
        qc.ch(q0, q1)
        qc.ry(theta, q0)

In [6]:
def initialize_blanks(qc, qr, mat):
    cands = get_cands(mat)
    # print(cands)

    for i in range(len(cands)):
        if len(cands[i]) == 1:
            single_cand_operation(qc, qr[2*i], qr[2*i+1], cands[i])
        elif len(cands[i]) == 2:
            two_cand_operation(qc, qr[2*i], qr[2*i+1], cands[i])
        else:
            three_cand_operation(qc, qr[2*i], qr[2*i+1], cands[i])

def initialize_blanks_inverse(qc, qr, mat):
    cands = get_cands(mat)
    
    for i in range(len(cands)-1, -1, -1):
        if len(cands[i]) == 1:
            single_cand_operation_inv(qc, qr[2*i], qr[2*i+1], cands[i])
        elif len(cands[i]) == 2:
            two_cand_operation_inv(qc, qr[2*i], qr[2*i+1], cands[i])
        else:
            three_cand_operation_inv(qc, qr[2*i], qr[2*i+1], cands[i])

In [7]:
# if v1 != v2 (number) then target = 1 else target = 0
def compare_vertex(qc, qr, v1, v2, target):
    qc.x(qr[2*v2])
    qc.x(qr[2*v2+1])
    qc.ccx(qr[2*v1], qr[2*v1+1], target)
    qc.ccx(qr[2*v1+1], qr[2*v2], target)
    qc.ccx(qr[2*v2], qr[2*v2+1], target)
    qc.ccx(qr[2*v1], qr[2*v2+1], target)
    qc.x(qr[2*v2+1])
    qc.x(qr[2*v2])
    #
    qc.x(qr[target])

def compare_vertex_inv(qc, qr, v1, v2, target):
    qc.x(qr[target])
    #
    qc.x(qr[2*v2])
    qc.x(qr[2*v2+1])
    qc.ccx(qr[2*v1], qr[2*v2+1], target)
    qc.ccx(qr[2*v2], qr[2*v2+1], target)
    qc.ccx(qr[2*v1+1], qr[2*v2], target)
    qc.ccx(qr[2*v1], qr[2*v1+1], target)
    qc.x(qr[2*v2+1])
    qc.x(qr[2*v2])

In [8]:
def diffusion(circuit, qr, mat):
    """
    diffusion (inversion about the mean) circuit.
    note that this implementation gives H^{\otimes n} (Id - |0..0><0...0|) H^{\otimes n}
    :param circuit:
    :param qr: QuantumRegister on nodes
    :return:
    """

    # circuit.h(qr)
    initialize_blanks_inverse(circuit, qr[:18], mat)  # 初期状態準備のユニタリ操作のinverse
    circuit.x(qr[:18])

    # apply multi-control CZ
    circuit.h(qr[17])
    # circuit.mct(qr[:-1], qr[-1], anc, mode='basic')
    circuit.mct(qr[:17], qr[17], qr[18:32], mode='advanced')  # ancilla使わないver
    circuit.h(qr[17])

    circuit.x(qr[:18])
    # circuit.h(qr)
    initialize_blanks(circuit, qr[:18], mat)

In [9]:
# とりあえず辺埋め込みオラクル
def simple_oracle(qc, qr):
    # vs = [0, 1, 2, 3, 4, 5, 6, 7]
    lst1 = [[0, 1], [2, 3], [4, 5], [6, 7], [0, 4], [1, 5]]
    lst2 = [[2, 6], [3, 7], [0, 2], [1, 3], [4, 6], [5, 7]]
    lst3 = [[1, 8], [5, 8], [6, 8], [7, 8]]
    target = 18
    tmp1 = 28
    tmp2 = 29
    tmp3 = 30
    for elm in lst1:
        assert(len(elm)==2)
        compare_vertex(qc, qr, elm[0], elm[1], target)
        target+=1
    qc.mct(qr[18:target], qr[tmp1], qr[24:28], mode='basic')
    for elm in (reversed(lst1)):
        target-=1
        compare_vertex_inv(qc, qr, elm[0], elm[1], target)
        
    for elm in lst2:
        assert(len(elm)==2)
        compare_vertex(qc, qr, elm[0], elm[1], target)
        target+=1
    qc.mct(qr[18:target], qr[tmp2], qr[24:28], mode='basic')
    for elm in (reversed(lst2)):
        target-=1
        compare_vertex_inv(qc, qr, elm[0], elm[1], target)
    
    for elm in lst3:
        assert(len(elm)==2)
        compare_vertex(qc, qr, elm[0], elm[1], target)
        target+=1
    qc.mct(qr[18:target], qr[tmp3], qr[24:28], mode='basic')
    for elm in (reversed(lst3)):
        target-=1
        compare_vertex_inv(qc, qr, elm[0], elm[1], target)
        
    ##
    dst = 31
    qc.mct(qr[tmp1:dst], qr[dst], [qr[18]], mode='basic')
    ##

    for elm in lst3:
        assert(len(elm)==2)
        compare_vertex(qc, qr, elm[0], elm[1], target)
        target+=1
    qc.mct(qr[18:target], qr[tmp3], qr[24:28], mode='basic')
    for elm in (reversed(lst3)):
        target-=1
        compare_vertex_inv(qc, qr, elm[0], elm[1], target)

    for elm in lst2:
        assert(len(elm)==2)
        compare_vertex(qc, qr, elm[0], elm[1], target)
        target+=1
    qc.mct(qr[18:target], qr[tmp2], qr[24:28], mode='basic')
    for elm in (reversed(lst2)):
        target-=1
        compare_vertex_inv(qc, qr, elm[0], elm[1], target)
        
    for elm in lst1:
        assert(len(elm)==2)
        compare_vertex(qc, qr, elm[0], elm[1], target)
        target+=1
    qc.mct(qr[18:target], qr[tmp1], qr[24:28], mode='basic')
    for elm in (reversed(lst1)):
        target-=1
        compare_vertex_inv(qc, qr, elm[0], elm[1], target)

In [10]:
qr = QuantumRegister(32)
cr = ClassicalRegister(18)
qc = QuantumCircuit(qr, cr)
mat = np.array([[1, 0, 4, 0],
              [0, 2, 0, 1],
              [2, 0, 1, 0],
              [0, 1, 0, 0]]).reshape(4, 4)
print(mat)
print(get_cands(mat))

def grover(iter):
    initialize_blanks(qc, qr, mat)
    for _ in range(iter):
        simple_oracle(qc, qr)
        print("simple oracle done")
        diffusion(qc, qr, mat)
        print("diffusion done")
    qc.measure(qr[0:18], cr[0:18])
    print("measure done")
    #print("drawing start")
    #qc.draw()

[[1 0 4 0]
 [0 2 0 1]
 [2 0 1 0]
 [0 1 0 0]]
[[3], [2, 3], [3, 4], [3], [3, 4], [3, 4], [3, 4], [2, 3], [2, 3, 4]]


In [11]:
grover(2)

import json
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(qc) 

# obtain gates
gates=new_circuit.count_ops()
print(gates)

cost=gates['u3'] + 10*gates['cx']
print(cost)


simple oracle done
diffusion done
simple oracle done
diffusion done
measure done
OrderedDict([('u3', 10544), ('cx', 5983), ('measure', 18)])
70374


In [ ]:
# shots = 8000
shots = 1000
print("shots:", shots)

## local simulator
#backend = Aer.get_backend('qasm_simulator')
## cloud simulator
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')

print("execute start")
job = execute(qc, backend=backend, shots=shots, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
count = result.get_counts()
print(count)

shots: 1000
execute start


In [ ]:
# 10進数(1~4)に変換
def convert_to_decimal(str_bin):
    str_bin = ''.join(list(reversed(str_bin)))  # cbitに格納されるときに順番逆になっていることに注意
    list_dec = [str(int(str_bin[2*i:2*i+2], 2)+1) for i in range(len(str_bin)//2)]
    str_dec = ''.join(list_dec)
    return str_dec
# histogram plot (decimal representation)
count_decimal = {}
for key, value in count.items():
    count_decimal[convert_to_decimal(key)] = value
print(count_decimal)
plot_histogram(count_decimal)


In [ ]:
score_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)
final_score = score_sorted[0:15]
final_score